# 🎓 AI융합캡스톤 디자인: RAG 기반 직무 추천 시스템 성능 검증

**팀명:** 베리타스 랩 (Veritas Lab)
**작성자:** 김혜성

---

### 📌 실험 개요
본 노트북은 **Basic LLM**과 **RAG(검색 증강 생성) LLM**의 직무 추천 성능을 비교합니다.

**활용 데이터 (공공데이터포털):**
1. **NCS 국가직무능력표준 정보**: 구체적인 수행 업무(Task) 매칭용
2. **한국고용정보원 직업세세분류**: 공식 직업 명칭(Job Title) 매칭용
3. **한국고용정보원 직업중분류**: 직업 카테고리 분류용

In [ ]:
!pip uninstall -y google-generativeai google-ai-generativelanguage langchain-google-genai

!pip install -U langchain-google-genai google-generativeai langchain langchain-community chromadb pandas

Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
Found existing installation: google-ai-generativelanguage 0.6.15
Uninstalling google-ai-generativelanguage-0.6.15:
  Successfully uninstalled google-ai-generativelanguage-0.6.15
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple v

In [ ]:
#Gemini 기반 RAG 시스템 구축
import os
import pandas as pd
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#구글 Gemini 라이브러리 임포트
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

os.environ['GOOGLE_API_KEY'] = '발급받은 키 입력'

## 2. 데이터 로드 및 전처리 (Data Ingestion)
세 개의 CSV 파일을 읽어 **'업무 내용(NCS)'**과 **'직업 명칭(고용정보원)'**을 통합한 지식 베이스를 구축합니다.

In [ ]:
FILE_NCS = 'ncs.csv'
FILE_JOB_DETAIL = 'job_detail.CSV'
FILE_JOB_MID = 'job_mid.CSV'

documents = []

print('🔄 데이터 로드 및 통합 중...')

# --- 1. NCS 데이터 처리 ---
try:
    df_ncs = pd.read_csv(FILE_NCS, encoding='cp949')
    # 필요한 컬럼만 추출하여 문서화
    for _, row in df_ncs.iterrows():
        # 문서 내용: [NCS] 업무명: ... (분류코드: ...)
        text = f"[NCS실무] 업무명: {row['명칭']} | NCS분류번호: {row['분류번호']}"
        documents.append(Document(page_content=text, metadata={'source': 'NCS'}))
    print(f'✅ NCS 데이터 처리 완료: {len(df_ncs)}개')
except Exception as e:
    print(f'⚠️ NCS 파일 로드 오류: {e}')

# --- 2. 직업분류 데이터 처리 (Join) ---
try:
    df_detail = pd.read_csv(FILE_JOB_DETAIL, encoding='cp949')
    df_mid = pd.read_csv(FILE_JOB_MID, encoding='cp949')

    # 대분류, 중분류 코드를 기준으로 병합 (Merge)
    # 공통 컬럼: KNOW직업대분류, KNOW직업중분류
    df_merged = pd.merge(df_detail, df_mid, on=['KNOW직업대분류', 'KNOW직업중분류'], how='left')

    for _, row in df_merged.iterrows():
        # 중분류명이 없는 경우 처리
        category = row['KNOW직업중분류명'] if pd.notna(row['KNOW직업중분류명']) else '기타'
        # 문서 내용: [직업사전] 직업명: ... (분류: ...)
        text = f"[공식직업명] 직업명: {row['KNOW직업명']} | 직업분류: {category}"
        documents.append(Document(page_content=text, metadata={'source': 'JobDictionary'}))

    print(f'✅ 직업분류 데이터 통합 완료: {len(df_merged)}개')

except Exception as e:
    print(f'⚠️ 직업분류 파일 로드 오류: {e}')

print(f'\n🎉 총 {len(documents)}개의 지식 데이터가 준비되었습니다.')

🔄 데이터 로드 및 통합 중...
✅ NCS 데이터 처리 완료: 13343개
✅ 직업분류 데이터 통합 완료: 537개

🎉 총 13880개의 지식 데이터가 준비되었습니다.


## 3. 벡터 데이터베이스 구축 (Embedding)
준비된 텍스트 데이터를 AI가 이해할 수 있는 벡터로 변환하여 ChromaDB에 저장합니다.

In [ ]:
# 벡터 DB 구축

if documents:
    print("⏳ 임베딩 및 벡터 저장소 생성 중... (데이터 양에 따라 시간 소요)")

    # 1. 임베딩
    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

    # 2. 벡터 DB
    vectorstore = Chroma.from_documents(documents, embeddings)

    # 3. [최종 튜닝] Retriever 설정 (광범위 검색)
    # fetch_k=100: 일단 유사한 문서 100개를 후보로 가져옴
    # k=20: 그 중 서로 다른 내용 20개를 최종 선별
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 20, 'fetch_k': 100, 'lambda_mult': 0.6}
    )

    print("✅ Vector DB 구축 완료! (광범위 검색 모드 적용: fetch_k=100)")
else:
    print("❌ 문서가 없습니다.")

⏳ 임베딩 및 벡터 저장소 생성 중... (데이터 양에 따라 시간 소요)
✅ Vector DB 구축 완료! (광범위 검색 모드 적용: fetch_k=100)


## 4. RAG vs Basic 성능 비교
동일한 질문에 대해 RAG 적용 전후의 답변 차이를 확인합니다.

In [ ]:
#사용 가능한 Gemini 모델 리스트 확인
import google.generativeai as genai
import os


genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

print("📋 사용 가능한 모델 목록:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(f" - {m.name}")

📋 사용 가능한 모델 목록:
 - models/gemini-2.5-pro-preview-03-25
 - models/gemini-2.5-flash
 - models/gemini-2.5-pro-preview-05-06
 - models/gemini-2.5-pro-preview-06-05
 - models/gemini-2.5-pro
 - models/gemini-2.0-flash-exp
 - models/gemini-2.0-flash
 - models/gemini-2.0-flash-001
 - models/gemini-2.0-flash-exp-image-generation
 - models/gemini-2.0-flash-lite-001
 - models/gemini-2.0-flash-lite
 - models/gemini-2.0-flash-lite-preview-02-05
 - models/gemini-2.0-flash-lite-preview
 - models/gemini-2.0-pro-exp
 - models/gemini-2.0-pro-exp-02-05
 - models/gemini-exp-1206
 - models/gemini-2.0-flash-thinking-exp-01-21
 - models/gemini-2.0-flash-thinking-exp
 - models/gemini-2.0-flash-thinking-exp-1219
 - models/gemini-2.5-flash-preview-tts
 - models/gemini-2.5-pro-preview-tts
 - models/learnlm-2.0-flash-experimental
 - models/gemma-3-1b-it
 - models/gemma-3-4b-it
 - models/gemma-3-12b-it
 - models/gemma-3-27b-it
 - models/gemma-3n-e4b-it
 - models/gemma-3n-e2b-it
 - models/gemini-flash-latest
 - mod

In [ ]:
#성능 비교 실험 (Dynamic RAG vs Basic LLM)

# 1. 모델 설정 (Gemini 2.0 Flash)
TARGET_MODEL = "gemini-2.0-flash"

try:
    llm = ChatGoogleGenerativeAI(model=TARGET_MODEL, temperature=0)

    # ---------------------------------------------------------
    # A. Basic LLM (비교군: 일반적인 답변)
    # ---------------------------------------------------------
    def ask_basic(question):
        prompt = ChatPromptTemplate.from_template(
            """
            사용자의 경험을 듣고 적합한 직무를 추천해줘.
            답변 시 ** 같은 마크다운 볼드체를 절대 사용하지 말고, 깔끔한 텍스트로만 작성해.

            사용자 입력: {question}
            """
        )
        chain = prompt | llm | StrOutputParser()
        return chain.invoke({"question": question})

    # ---------------------------------------------------------
    # B. RAG LLM (실험군: 동적 쿼리 + NCS 데이터 기반)
    # ---------------------------------------------------------
    def ask_rag(question):
        if 'retriever' not in globals(): return "❌ 오류: 벡터 DB가 없습니다."

        # [Step 1] 동적 쿼리 생성 (Dynamic Query Generation)
        # 사용자의 질문을 검색하기 좋은 NCS 전문 용어로 변환
        keyword_prompt = ChatPromptTemplate.from_template(
            """
            사용자의 입력에서 직무 검색에 필요한 핵심 키워드 5개를 추출해줘.
            (예: 파이썬 -> 소프트웨어, 프로그래밍, 빅데이터)
            결과는 쉼표로 구분된 단어만 출력해.

            사용자 입력: {question}
            """
        )
        keyword_chain = keyword_prompt | llm | StrOutputParser()
        generated_keywords = keyword_chain.invoke({"question": question})

        # 디버깅: 내부적으로 어떤 키워드를 만들었는지 출력
        print(f"   └─ 🧠 AI가 생성한 검색 키워드: [{generated_keywords}]")

        # 검색어 확장
        enhanced_query = f"{question} {generated_keywords}"

        # [Step 2] 검색 및 답변 생성
        docs = retriever.invoke(enhanced_query)
        context_text = "\n\n".join([d.page_content for d in docs])

        template = """
        당신은 국가직무능력표준(NCS) 기반 커리어 컨설턴트입니다.
        [참고 문서]를 바탕으로 사용자에게 적합한 직무를 추천하세요.

        [지시사항]
        1. [참고 문서]에 있는 직무 중, 사용자 경험과 가장 논리적으로 연결되는 것을 찾으세요.
        2. 기계/제조/단순노무 관련 내용은 무시하고, 사용자의 전문성(키워드)에 맞는 직무만 추천하세요.
        3. 답변에 ** 같은 마크다운 볼드체를 절대 사용하지 마세요. 가독성 좋게 번호를 매겨서 작성하세요.

        [참고 문서]:
        {context}

        사용자 입력: {question}

        [답변 양식]
        1. 추천 직무: [공식 직업 명칭]
        2. NCS 매칭 근거:
           - 관련 업무: [NCS 업무명] (분류번호)
           - 분석: [사용자 경험과 NCS 업무의 연결고리 설명]
        3. 종합 의견: (Dynamic RAG 분석 결과)
        """

        prompt = ChatPromptTemplate.from_template(template)
        chain = prompt | llm | StrOutputParser()

        return chain.invoke({"context": context_text, "question": question})

    # ---------------------------------------------------------
    # [실험 실행] 사용자 페르소나 입력
    # ---------------------------------------------------------
    user_persona = """
    저는 파이썬 코딩을 좋아하고, 데이터를 엑셀로 정리해서 친구들에게 보여주는 걸 잘해요.
    자동화 프로그램 만드는 것에 관심이 많습니다.
    """

    print(f"👤 사용자 경험 입력:\n{user_persona.strip()}")
    print("="*60)

    # 1. Basic 모델 결과
    print("\n🤖 [Basic LLM] 일반적인 답변:")
    print("-" * 30)
    print(ask_basic(user_persona).strip())

    print("\n" + "="*60 + "\n")

    # 2. RAG 모델 결과
    print("🚀 [RAG LLM] NCS 데이터 기반 답변:")
    print("-" * 30)
    print(ask_rag(user_persona).strip())
    print("\n" + "="*60)

except Exception as e:
    print(f"❌ 오류 발생: {e}")

👤 사용자 경험 입력:
저는 파이썬 코딩을 좋아하고, 데이터를 엑셀로 정리해서 친구들에게 보여주는 걸 잘해요.
    자동화 프로그램 만드는 것에 관심이 많습니다.

🤖 [Basic LLM] 일반적인 답변:
------------------------------
파이썬 코딩 능력과 엑셀 데이터 정리 경험, 그리고 자동화 프로그램에 대한 관심사를 바탕으로 몇 가지 직무를 추천해 드립니다.

1. 데이터 분석가: 파이썬을 활용하여 데이터를 분석하고, 엑셀로 결과를 정리하는 능력은 데이터 분석가에게 매우 중요합니다. 자동화 프로그램을 통해 데이터 수집 및 분석 과정을 효율적으로 만들 수 있습니다.

2. 백엔드 개발자: 파이썬은 백엔드 개발에 많이 사용되는 언어입니다. 자동화된 시스템 구축 경험을 바탕으로 서버 로직 개발, API 개발 등의 업무를 수행할 수 있습니다.

3. RPA(Robotic Process Automation) 개발자: 자동화 프로그램 개발에 대한 관심은 RPA 개발 직무와 잘 맞습니다. 파이썬을 사용하여 반복적인 업무를 자동화하는 프로그램을 개발하고 유지보수하는 역할을 합니다.

4. 기술 지원 엔지니어: 파이썬 스크립트를 활용하여 고객 문제 해결을 자동화하거나, 엑셀을 활용하여 고객 데이터를 분석하고 리포트를 작성하는 데 기여할 수 있습니다.

5. 엑셀 자동화 개발자: 엑셀 VBA 또는 파이썬을 사용하여 엑셀 작업을 자동화하는 프로그램을 개발하는 직무입니다. 엑셀 데이터 정리 경험과 자동화에 대한 관심사를 직접적으로 활용할 수 있습니다.

이 외에도 파이썬과 엑셀 활용 능력을 필요로 하는 다양한 직무가 있습니다. 본인의 관심사와 추가적인 학습 계획 등을 고려하여 적합한 직무를 선택하시면 좋겠습니다.


🚀 [RAG LLM] NCS 데이터 기반 답변:
------------------------------
   └─ 🧠 AI가 생성한 검색 키워드: [소프트웨어, 프로그래밍, 데이터 분석, 자동화, 엑셀]
1. 추천 직무: 자동 배치배선 레이

### 🧪 실험 실행 (Simulation)
아래에 다양한 사용자 페르소나를 입력하여 결과를 비교해보세요.

In [ ]:
# [테스트 케이스] 컴퓨터공학과 학생의 경험
user_persona = """
저는 파이썬을 이용해서 웹 크롤링을 해본 적이 있고,
수집한 데이터를 엑셀로 정리해서 친구들에게 공유해준 경험이 있습니다.
새로운 기술을 배우는 걸 좋아하고, 반복되는 작업을 자동화하는 데 흥미가 있어요.
"""

print(f'👤 사용자 입력: {user_persona.strip()}\n' + '='*70)

print('🤖 [Basic LLM 답변] (일반적 지식)')
print(ask_basic(user_persona))
print('\n' + '-'*70 + '\n')

print('🚀 [RAG LLM 답변] (NCS & 직업사전 데이터 기반)')
print(ask_rag(user_persona))
print('='*70)

👤 사용자 입력: 저는 파이썬을 이용해서 웹 크롤링을 해본 적이 있고, 
수집한 데이터를 엑셀로 정리해서 친구들에게 공유해준 경험이 있습니다.
새로운 기술을 배우는 걸 좋아하고, 반복되는 작업을 자동화하는 데 흥미가 있어요.
🤖 [Basic LLM 답변] (일반적 지식)
파이썬을 이용한 웹 크롤링 경험과 엑셀 데이터 정리 경험, 그리고 새로운 기술 학습에 대한 흥미와 자동화에 대한 관심을 바탕으로 몇 가지 적합한 직무를 추천해 드립니다.

1. 데이터 분석가: 웹 크롤링으로 수집한 데이터를 분석하고, 엑셀을 활용하여 정리하는 경험은 데이터 분석가에게 필요한 기본적인 역량입니다. 새로운 기술 학습에 대한 흥미는 데이터 분석 도구 및 통계 기법을 배우고 적용하는 데 도움이 될 것입니다.

2. 백엔드 개발자: 파이썬은 백엔드 개발에 널리 사용되는 언어입니다. 웹 크롤링 경험은 웹 서비스의 작동 방식에 대한 이해를 높여주며, 자동화에 대한 관심은 서버 관리 및 배포 자동화에 기여할 수 있습니다.

3. RPA(Robotic Process Automation) 개발자: 반복적인 작업을 자동화하는 데 흥미를 느끼신다면 RPA 개발자가 적합할 수 있습니다. 파이썬을 이용하여 업무 프로세스를 자동화하는 로봇을 개발하고 유지보수하는 역할을 수행합니다.

4. 기술 지원 엔지니어: 기술적인 문제 해결 능력과 새로운 기술 학습에 대한 흥미를 바탕으로 고객에게 기술적인 지원을 제공하는 역할을 수행할 수 있습니다. 웹 크롤링 및 데이터 처리 경험은 문제 해결에 도움이 될 것입니다.

5. 데이터 엔지니어: 대용량 데이터를 수집, 저장, 처리, 분석할 수 있도록 데이터 파이프라인을 구축하고 관리하는 역할을 합니다. 웹 크롤링 경험은 데이터 수집 과정에 대한 이해를 높여주며, 파이썬은 데이터 엔지니어링에 필수적인 언어입니다.

이 외에도 데이터 관련 직무, 자동화 관련 직무 등 다양한 분야에서 사용자님의 경험과 흥미를 활용할 수 있습니다.

------------------------

In [ ]:
#다양한 도메인에 대한 RAG 반응성 테스트

import time

# 1. 테스트할 시나리오 목록 (다양한 페르소나)
test_scenarios = [
    {
        "category": "🎨 [1. 디자인/마케팅 분야]",
        "question": "저는 그림 그리는 걸 좋아하고 포토샵으로 포스터 만드는 게 재밌어요. 사람들의 시선을 끄는 창의적인 광고를 만들고 싶어요."
    },
    {
        "category": "💪 [2. 스포츠/헬스 분야]",
        "question": "몸 쓰는 걸 좋아하고 남들에게 운동 가르쳐주는 걸 잘합니다. 활동적인 직업을 갖고 싶고 건강 관리에 관심이 많아요."
    },
    {
        "category": "🔧 [3. 기계/설비 분야 (필터링 검증)]",
        "question": "손으로 무언가 조립하고 기계를 고치는 걸 잘해요. 전선 연결이나 배선 작업, 납땜 같은 정밀한 작업에 흥미가 있습니다."
    },
    {
        "category": "💻 [4. SW 개발 분야 (Control Group)]",
        "question": "자바(Java)랑 스프링 부트(Spring Boot)를 공부했고, 웹사이트 회원가입 기능을 직접 구현해본 적이 있습니다. 백엔드 개발이 하고 싶어요."
    }
]

print(f"🧪 총 {len(test_scenarios)}개의 시나리오에 대해 테스트를 시작합니다...")
print("="*70)

# 2. 반복 테스트 실행
for idx, scenario in enumerate(test_scenarios):
    category = scenario["category"]
    question = scenario["question"]

    print(f"\n{category}")
    print(f"❓ 질문: {question}")
    print("-" * 50)

    try:
        # Basic 결과
        print("🤖 [Basic LLM]:")
        # 줄바꿈 제거하고 앞부분만 간략히 출력 (너무 길어지는 것 방지)
        basic_ans = ask_basic(question).replace("\n", " ")[:150] + "..."
        print(f"{basic_ans}")

        print("-" * 20)

        # RAG 결과 (전체 출력)
        print("🚀 [RAG LLM]:")
        rag_ans = ask_rag(question).strip()
        print(rag_ans)

    except Exception as e:
        print(f"❌ 에러 발생: {e}")

    print("="*70)
    # API 호출 제한 방지를 위해 약간의 딜레이
    time.sleep(2)


🧪 총 4개의 시나리오에 대해 테스트를 시작합니다...

🎨 [1. 디자인/마케팅 분야]
❓ 질문: 저는 그림 그리는 걸 좋아하고 포토샵으로 포스터 만드는 게 재밌어요. 사람들의 시선을 끄는 창의적인 광고를 만들고 싶어요.
--------------------------------------------------
🤖 [Basic LLM]:
그림 그리는 것을 좋아하고 포토샵을 활용한 포스터 제작 경험이 있으며, 사람들의 시선을 끄는 창의적인 광고 제작에 흥미를 느끼시는군요. 이러한 경험과 관심사를 바탕으로 다음과 같은 직무를 추천해 드립니다.  1.  광고 디자이너: 광고 캠페인의 시각적인 요소를 디자인하...
--------------------
🚀 [RAG LLM]:
   └─ 🧠 AI가 생성한 검색 키워드: [디자인, 포토샵, 광고, 창의, 시각]
제공된 NCS 실무 목록에서 사용자 경험과 관련된 직무는 직접적으로 드러나지 않습니다. 하지만 사용자의 "그림 그리는 것을 좋아하고 포토샵으로 포스터 만드는 것이 재밌다"는 경험과 "사람들의 시선을 끄는 창의적인 광고를 만들고 싶다"는 목표를 고려하여 가장 연관성이 높은 직무를 추론해 보겠습니다.

1. 추천 직무: (제공된 NCS 목록 내에서는 직접적인 매칭이 어려움) 광고/홍보 전문가 (유사 직무)
2. NCS 매칭 근거:
   - 관련 업무: 디지털 방사선 영상 판독 (2306030805_20v1)
   - 분석: 디지털 방사선 영상 판독은 의료 영상 이미지를 분석하여 질병 유무를 판단하는 업무입니다. 사용자의 포토샵 경험과 시각적인 결과물을 만들어내는 능력은 영상 판독 과정에서 이미지 편집 및 분석 도구를 활용하는 데 도움이 될 수 있습니다. 또한, "사람들의 시선을 끄는" 능력은 영상 판독 결과의 시각적 표현에 긍정적인 영향을 미칠 수 있습니다.
3. 종합 의견: 제공된 NCS 목록은 주로 기술 및 제조 분야에 집중되어 있어 사용자의 창의적인 광고 제작 역량과 직접적으로 연결되는 직무를 찾기 어렵습니다.

In [ ]:
#STAR 기법 기반 고품질 역량 매칭 테스트

import time

# STAR 기법이 적용된 상세 시나리오 (MVP에서 추출될 데이터 예시)
star_scenarios = [
    {
        "role": "🧑‍💻 [Case 1: 백엔드 개발 경험 (SW)]",
        "star_story": """
        [Situation] 학부 수강신청 기간에 트래픽이 몰려 서버가 다운되는 문제가 있었습니다.
        [Task] 서버의 부하를 줄이고 동시 접속자를 처리해야 하는 목표를 가졌습니다.
        [Action] 저는 Python Django 프레임워크를 기반으로 Redis 캐싱을 도입하여 DB 조회를 줄였고, 비동기 큐(Celery)를 사용하여 대기열 시스템을 구현했습니다.
        [Result] 그 결과 서버 응답 속도를 50% 단축시켰고, 1,000명의 동시 접속을 안정적으로 처리했습니다.
        """
    },
    {
        "role": "📊 [Case 2: 데이터 분석 경험 (SW/BigData)]",
        "star_story": """
        [Situation] 교내 창업 동아리에서 마케팅 효율이 떨어지는 원인을 파악해야 했습니다.
        [Task] 고객 구매 데이터를 분석하여 타겟 고객층을 재정의해야 했습니다.
        [Action] 엑셀에 있던 1만 건의 raw 데이터를 Python Pandas로 전처리하고, Scikit-learn의 K-means 클러스터링 알고리즘을 사용해 고객 군집을 3개로 분류했습니다.
        [Result] 이를 통해 20대 여성 타겟층을 발견하여 마케팅 적중률을 15% 상승시켰습니다.
        """
    },
    {
        "role": "🎨 [Case 3: UI/UX 디자인 경험 (Design/SW)]",
        "star_story": """
        [Situation] 개발 중인 앱의 사용자 이탈률이 높다는 피드백을 받았습니다.
        [Task] 사용자 인터페이스(UI)를 직관적으로 개선하여 체류 시간을 늘려야 했습니다.
        [Action] Figma를 사용하여 와이어프레임을 다시 설계했고, A/B 테스트를 진행하여 버튼 위치와 색상을 최적화했습니다. 또한 프론트엔드 개발자와 협업하여 CSS 애니메이션을 적용했습니다.
        [Result] 앱의 일일 활성 사용자(DAU)가 2배 증가하는 성과를 거두었습니다.
        """
    },
    {
        "role": "🔧 [Case 4: 기계 설비 유지보수 (Non-SW Control Group)]",
        "star_story": """
        [Situation] 공장 아르바이트 중 컨베이어 벨트 모터에서 이상 소음이 발생했습니다.
        [Task] 라인이 멈추기 전에 원인을 파악하고 수리해야 했습니다.
        [Action] 회로도를 분석하여 전압 불균형 문제를 찾아냈고, 납땜 장비를 이용해 배선을 다시 연결한 뒤 윤활유를 도포하여 기어 마찰을 줄였습니다.
        [Result] 30분 만에 설비를 정상화하여 생산 지연을 막았습니다.
        """
    }
]

print(f"🧪 STAR 기법 기반 총 {len(star_scenarios)}개의 시나리오 테스트 시작...")
print("="*80)

for idx, scenario in enumerate(star_scenarios):
    role = scenario["role"]
    story = scenario["star_story"]

    print(f"\n{role}")
    print(f"📝 경험 기술(STAR): {story.strip()}")
    print("-" * 60)

    try:
        # Basic LLM 결과 (요약 출력)
        print("🤖 [Basic LLM 답변]:")
        basic_ans = ask_basic(story).replace("\n", " ")
        print(f"{basic_ans[:150]}...") # 너무 길면 자름

        print("-" * 30)

        # RAG LLM 결과 (전체 출력)
        print("🚀 [RAG LLM 답변] (NCS 매칭):")
        rag_ans = ask_rag(story).strip()
        print(rag_ans)

    except Exception as e:
        print(f"❌ 에러 발생: {e}")

    print("="*80)
    time.sleep(3) # API 호출 딜레이

🧪 STAR 기법 기반 총 4개의 시나리오 테스트 시작...

🧑‍💻 [Case 1: 백엔드 개발 경험 (SW)]
📝 경험 기술(STAR): [Situation] 학부 수강신청 기간에 트래픽이 몰려 서버가 다운되는 문제가 있었습니다.
        [Task] 서버의 부하를 줄이고 동시 접속자를 처리해야 하는 목표를 가졌습니다.
        [Action] 저는 Python Django 프레임워크를 기반으로 Redis 캐싱을 도입하여 DB 조회를 줄였고, 비동기 큐(Celery)를 사용하여 대기열 시스템을 구현했습니다.
        [Result] 그 결과 서버 응답 속도를 50% 단축시켰고, 1,000명의 동시 접속을 안정적으로 처리했습니다.
------------------------------------------------------------
🤖 [Basic LLM 답변]:
제공해주신 경험을 바탕으로 볼 때, 서버 성능 개선 및 시스템 안정화에 기여한 경험이 돋보입니다. 특히, 문제 해결 능력, 기술적인 이해도, 그리고 실제 적용 능력이 뛰어난 것으로 판단됩니다.  다음과 같은 직무를 추천합니다.  *   백엔드 개발자: 서버 측 로직 개...
------------------------------
🚀 [RAG LLM 답변] (NCS 매칭):
   └─ 🧠 AI가 생성한 검색 키워드: [Python, Django, Redis, Celery, 서버]
1. 추천 직무: 사업 환경 분석가

2. NCS 매칭 근거:
   - 관련 업무: 사업 환경 분석 (2305060101_15v1)
   - 분석: 사용자가 제시한 상황은 수강신청 시스템의 트래픽 과부하 문제이며, 이는 사업 환경 분석에서 다루는 '내부 환경 분석'과 연결될 수 있습니다. 사용자는 문제 해결을 위해 기술적인 솔루션을 적용했지만, 근본적으로는 시스템이 운영되는 환경(트래픽 패턴, 사용자 행동 등)을 분석하고 이해하는 것이 중요합니다. 사업 환경 분석가는 이러한 환경 분석을 통해 문제의 원인